In [1]:
import joblib
from dask.distributed import Client
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Group 7 - Recommendation System')\
.config('spark.sql.execution.arrow.pyspark.enabled', True)\
.config('spark.driver.memory','8G')\
.config('spark.ui.showConsoleProgress', True)\
.config('spark.sql.repl.eagerEval.enabled', True)\
.getOrCreate()

client = Client()

21/07/16 23:48:44 WARN Utils: Your hostname, zas resolves to a loopback address: 127.0.1.1; using 192.168.1.225 instead (on interface wlp0s20f3)
21/07/16 23:48:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/16 23:48:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Data is downloaded from https://www.kaggle.com/bandikarthik/movie-recommendation-system
movies = spark.read.csv('../MovieLens/movie.csv', header=True, inferSchema=True)
ratings = spark.read.csv('../MovieLens/rating.csv',  header=True, inferSchema=True)

In [3]:
with joblib.parallel_backend('dask'):
    movies_df = movies.toPandas()
    rating_df = ratings.toPandas()

In [5]:
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
movies_users= rating_df.head(1000000).pivot(index='movieId', columns='userId',values='rating').fillna(0)

In [6]:
with joblib.parallel_backend('dask'):
    mat_movies_users=csr_matrix(movies_users.values)
    model_knn.fit(mat_movies_users)

In [11]:
from fuzzywuzzy import process
def recommender(movie_name, data, model, n_recommendations ):
    df_movies = movies.toPandas()
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected: ', df_movies['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))

movie = "Heavy (1995)"
print("Recommend movies for people watched " + movie)
recommender(film, mat_movies_users, model_knn, 10)

Recommend movies for people watched Heavy (1995)
Movie Selected:  Heavy (1995) Index:  751
Searching for recommendations.....
751                                                NaN
628                             Family Thing, A (1996)
709           Halfmoon (Paul Bowles - Halbmond) (1995)
470                         In the Line of Fire (1993)
706              Visitors, The (Visiteurs, Les) (1993)
254                                  Just Cause (1995)
0                                     Toy Story (1995)
1155                             Paths of Glory (1957)
1489    Second Jungle Book: Mowgli & Baloo, The (1997)
577                    Dear Diary (Caro Diario) (1994)
Name: title, dtype: object
